### Import pakietów

In [42]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Proces ekstrakcji opinii o produkcie ze strony Ceneo.pl

In [43]:
with open("./headers.json", "r") as jf:
    headers = json.load(jf)

In [44]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
    
while next_page:

    print(next_page)
    
    response = requests.get(next_page, headers=headers)

    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

        for opinion in opinions:
            try:
                single_opinion = {
                    "author" : opinion.select_one("span.user-post__author-name").text.strip(),
                    "recommendation" :  opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
                    "stars" : opinion.select_one("span.user-post__score-count").text.strip(),
                    "content" : opinion.select_one("div.user-post__text").text.strip(),
                    "pros" : [p.text.strip() for p in opinion.select("div.review-feature__item--positive")],
                    "cons" : [c.text.strip() for c in opinion.select("div.review-feature__item--negative")],
                    "useful" : opinion.select_one("button.vote-yes > span").text.strip(),
                    "unuseful" : opinion.select_one("button.vote-no > span").text.strip(),
                    "post_date" : opinion.select_one("span.user-post__published > time:nth-of-type(1)")['datetime'].strip(),
                    "purchase_date" : opinion.select_one("span.user-post__published > time:nth-of-type(2)")['datetime'].strip(),
                }
                
                all_opinions.append(single_opinion)
            except (AttributeError, TypeError):
                pass

            try:
                next_page = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
            except TypeError:
                next_page = None
    else: print(response.status_code)


https://www.ceneo.pl/84514582#tab=reviews
https://www.ceneo.pl/84514582/opinie-2
https://www.ceneo.pl/84514582/opinie-3
https://www.ceneo.pl/84514582/opinie-4
https://www.ceneo.pl/84514582/opinie-5
https://www.ceneo.pl/84514582/opinie-6
https://www.ceneo.pl/84514582/opinie-7
https://www.ceneo.pl/84514582/opinie-8
https://www.ceneo.pl/84514582/opinie-9


### Zapisanie wyników do bazy danych

In [45]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [46]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)